## Noof

The `NoofModel` is a model that uses a transformer on the embeddings. 
It's pretty terrible.

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import torch.nn as nn
from torch.nn import TransformerEncoder, TransformerEncoderLayer

from netam import framework, models
from netam.common import pick_device, PositionalEncoding

from shmex.shm_data import train_val_dfs_of_nickname

In [2]:
kmer_length = 3
site_count = 410

train_df, val_df = train_val_dfs_of_nickname('shmoof_small')

train_dataset = framework.SHMoofDataset(train_df, kmer_length=kmer_length, site_count=site_count)
val_dataset = framework.SHMoofDataset(val_df, kmer_length=kmer_length, site_count=site_count)

device = pick_device()
train_dataset.to(device)
val_dataset.to(device)

print(f"we have {len(train_dataset)} training examples and {len(val_dataset)} validation examples")

Interpreting shmoof_small as a shmoof dataset
Using Metal Performance Shaders
we have 46391 training examples and 2625 validation examples


In [7]:
class NoofModel(models.KmerModel):
    def __init__(
        self, kmer_length, embedding_dim, nhead, dim_feedforward, layer_count, dropout_prob=0.3
    ):
        super().__init__(kmer_length)
        self.embedding_dim = embedding_dim

        self.kmer_embedding = nn.Embedding(self.kmer_count, embedding_dim)
        self.pos_encoder = PositionalEncoding(embedding_dim, dropout=dropout_prob)

        self.encoder_layer = TransformerEncoderLayer(
            d_model=self.embedding_dim,
            nhead=nhead,
            dim_feedforward=dim_feedforward,
            dropout=dropout_prob,
            batch_first=True,
        )
        self.encoder = TransformerEncoder(self.encoder_layer, layer_count)
        self.linear = nn.Linear(self.embedding_dim, 1)

        self.init_weights()

    def init_weights(self) -> None:
        initrange = 0.1
        self.linear.bias.data.zero_()
        self.linear.weight.data.uniform_(-initrange, initrange)

    def forward(self, encoded_parents, masks, wt_base_modifier):
        """
        The forward method.

        encoded_parents is expected to be an integer tensor of [batch_size, sequence_length].
        """
        kmer_embeddings = self.kmer_embedding(encoded_parents)
        kmer_embeddings = self.pos_encoder(kmer_embeddings)

        # Pass through the transformer encoder
        transformer_output = self.encoder(kmer_embeddings)

        # Apply the linear layer and squeeze out the last dimension.
        # After the linear layer, the dimensions will be [batch_size, sequence_length, 1].
        # We squeeze out the last dimension to make it [batch_size, sequence_length].
        log_rates = self.linear(transformer_output).squeeze(-1)
        rates = torch.exp(log_rates)
        return rates



In [8]:
model = NoofModel(kmer_length=kmer_length, embedding_dim=10, nhead=2, dim_feedforward=256, layer_count=2, dropout_prob=0.1)

device = pick_device()
train_dataset.to(device)
val_dataset.to(device)
model.to(device)

burrito = framework.SHMBurrito(train_dataset, val_dataset, model, batch_size=1024, learning_rate=0.001, weight_decay=1e-6)
print("starting training...")
losses = burrito.train(epochs=100)
losses

Using Metal Performance Shaders
starting training...


Epoch:  57%|█████▋    | 57/100 [1:01:51<46:39, 65.11s/it, loss_diff=2.161e-07, lr=6.25e-5, val_loss=0.05661]  


,train_loss,val_loss
0,0.064748,0.056811
1,0.064110,0.056720
2,0.063948,0.056638
3,0.063859,0.056569
4,0.063806,0.056598
5,0.063779,0.056607
6,0.063753,0.056598
7,0.063719,0.056609
8,0.063701,0.056592
9,0.063694,0.056598
